![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use Watsonx to analyze customer review comments to gain insights inorder to determine improvements in product

**Note:** Please note that for the watsonx challenge, please run these notebooks in IBM Cloud and not on on your laptop/desktop.

This notebook contains the steps and code to demonstrate support of text summarization in Watsonx. It introduces commands for data retrieval, model testing and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.10.

<a id="setup"></a>
## Set up the environment

### Install and import the dependecies

In [ ]:
!pip install datasets | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install ibm-watson-machine-learning==1.0.349 | tail -n 1
!pip install rouge_score
!pip install evaluate

In [ ]:
import os, getpass
from pandas import read_csv
import evaluate

### Watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. Instructions have been provided to generate IBM Cloud API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [ ]:
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, BearerTokenAuthenticator
import os, getpass

access_token = IAMTokenManager(
    apikey = getpass.getpass("Please enter your api key (hit enter): "),
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

### Defining the project id
The API requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. When you run notebook on IBM Cloud, project in which it runs is saved as environment variable PROJECT_ID.

**Hint**: You can find the `project_id` as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be `Projects / <project name> /`. Click on the `<project name>` link. Then get the `project_id` from Project's Manage tab (Project -> Manage -> General -> Details).


In [ ]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="data"></a>
## Train/test data loading

Load train and test datasets. At first, training dataset (`train_data`) should be used to work with the models to prepare and tune prompt. Then, test dataset (`test_data`) should be used to calculate the metrics score for selected model, defined prompts and parameters.

In [ ]:
filename_test = 'https://watsonx-challenge-2024.s3.us.cloud-object-storage.appdomain.cloud/test.csv'
filename_train = 'https://watsonx-challenge-2024.s3.us.cloud-object-storage.appdomain.cloud/train.csv'

test_data = read_csv(filename_test)
train_data = read_csv(filename_train)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

<a id="models"></a>
## Foundation Models on Watsonx



Below code invokes Watson Machine Learning API to invoke Watsonx.ai LLMs


In [ ]:
import requests

class Prompt:
    def __init__(self, access_token, project_id):
        self.access_token = access_token
        self.project_id = project_id

    def generate(self, input, model_id, parameters):
        wml_url = "https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text?version=2023-05-28"
        Headers = {
            "Authorization": "Bearer " + self.access_token,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        data = {
            "model_id": model_id,
            "input": input,
            "parameters": parameters,
            "project_id": self.project_id
        }
        response = requests.post(wml_url, json=data, headers=Headers)
        if response.status_code == 200:
            return response.json()["results"][0]["generated_text"]
        else:
            return response.text

<a id="predict"></a>
## Evaluate the model, prompt and parameters

### **Analyze Product Reviews**

Define instructions for the model to summarize product reviews.

**Note:** Please **start with using [watsonx.ai Prompt Lab](https://dataplatform.cloud.ibm.com/wx/home?context=wx)** to find better prompts that provides you the best result on a small subset training records (under `train_data` variable). Make sure to not run an inference of all of `train_data`, as it'll take a long time to get the results. To get a sample from `train_data`, you can use e.g.`train_data.head(n=10)` to get first 10 records, or `train_data.sample(n=10)` to get random 10 records. Only once you have identified the best performing prompt, update this notebook to use the prompt and compute the metrics on the test data.

**Action:** Please edit the below cell and add your own prompt here. In the below prompt, we have the instruction (first sentence) and tags like [Document], [End] and <|assistant|> which help to organize inputs and outputs in a better way. (reference - https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models-ibm-chat.html?context=wx)

It will act as a starting point for the participants. You will enhance it to get expected output.

Change the prompt or add your own examples or more examples and replance it in the below cell.


**Note:** In below prompt {reviews} needs to be replaced with input product review that needs to be summarised. Rest all is part of the prompt.

In [ ]:
summarize_instruction = """

You are a retail expert. You will be given a list of product reviews. Your task is to generate a short summary, only from given product reviews from an ecommerce site to give feedback to another customer. When generating responses, prioritize correctness, i.e., ensure that your response is correct given the context and user query, and that it is grounded in the context.

[Document]
{reviews}
[End]


<|assistant|>

"""

### Defining the model parameters
We need to provide a set of model parameters that will influence the result. We will use IBM's Granite model. parameters can be updated based on prompt.

In [ ]:
parameters = {
    "decoding_method": "greedy",""
    "max_new_tokens": 300,
    "min_new_tokens": 50,
    "repetition_penalty": 1
}

model_id = "ibm/granite-13b-chat-v2"

Analyze the summary of product reviews for inputs from the test set.

**Note:** Execution of this cell could take several minutes.

In [ ]:
results = []
prompt = Prompt(access_token, project_id)
product_reviews = list(test_data.reviews)
summary = list(test_data.summary.astype(str))

for review in product_reviews:
    # Below line of code replaces {reviews} from prompt with input product review that needs to be summarised. You might need to change or replace it based on your prompt to add review.
    prompt_instruction = summarize_instruction.format(reviews=review)
    results.append(prompt.generate(prompt_instruction, model_id, parameters).replace("\n",""))

### Calculate the Rouge Scores

In [ ]:
# load the rouge score logic for summarization evaluation
# To adjust this to different task other than summarization import different evalualtion metric from HF
rouge_scorer_hf = evaluate.load('rouge')

In [ ]:
# test score: 50% of the words from the reference summary occur in the generated summary - should be rouge1 score should be 0.5
rouge_scores = rouge_scorer_hf.compute(predictions = results, 
                        references = summary)

In [ ]:
print(rouge_scores['rouge1'])